   # Recognition Model 

# Installation Part

You have to intall some of the libraries which are not already in Jupiter 

In [10]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


# Import Libraries

Here import all the useful libraries and loading the MNIST dataset

In [14]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

(x_train, y_train), (x_test, y_test) = mnist.load_data()   #load the mnist dataset
print(x_train.shape, y_train.shape)

11493376/11490434 [==============================] - 27s 2us/step
(60000, 28, 28) (60000,)


# Data Handling/Preprocessing

Before coming to the training part we will initialize our dataset and make it able to run through the CNN model

We have 60000 training samples and 10000 testing samples..

In [17]:
num_classes = 10
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)     #adding 1 more dimention to the data to make it for CNN model
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255      #normalization of data
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


# CNN Model

We are going to create our CNN model, we use to perform pooling operation which outputs a downsized version of the input.

We also use ReLu function -which is used to provide output directly if input is positive.

And we will drop the some neurons which are just making the condition of overfitting

In [18]:
batch_size = 128      #number of training examples in one batch
num_classes = 10
epochs = 10
model = Sequential()    #appropriate for a plain stack of layers where each layer has exactly one input tensor and one output tensor.
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))  #activating relu function
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))  #here max pooling used
model.add(Dropout(0.25))  #droptout neurons
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])

# Training Part

This is most important part ,it will take some time to train the data , uses training data to train the model.

model.fit() is used for training the model

In [19]:
#model.fit() is used to train the data
hist = model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test, y_test))
print("The model has successfully trained")
model.save('mnist.h5')
print("Saving the model as mnist.h5")

Epoch 1/10
469/469 [==============================] - 70s 149ms/step - loss: 2.2743 - accuracy: 0.1703 - val_loss: 2.2255 - val_accuracy: 0.4169
Epoch 2/10
469/469 [==============================] - 72s 154ms/step - loss: 2.1935 - accuracy: 0.3339 - val_loss: 2.1237 - val_accuracy: 0.6187
Epoch 3/10
469/469 [==============================] - 73s 155ms/step - loss: 2.0835 - accuracy: 0.4618 - val_loss: 1.9799 - val_accuracy: 0.6873
Epoch 4/10
469/469 [==============================] - 72s 153ms/step - loss: 1.9321 - accuracy: 0.5457 - val_loss: 1.7835 - val_accuracy: 0.7277
Epoch 5/10
469/469 [==============================] - 70s 150ms/step - loss: 1.7326 - accuracy: 0.6000 - val_loss: 1.5448 - val_accuracy: 0.7616
Epoch 6/10
469/469 [==============================] - 76s 161ms/step - loss: 1.5167 - accuracy: 0.6350 - val_loss: 1.2973 - val_accuracy: 0.7841
Epoch 7/10
469/469 [==============================] - 76s 161ms/step - loss: 1.3102 - accuracy: 0.6692 - val_loss: 1.0800 - val_ac

# Finally come to Testing Part

This will show the acuracy of our model

In [20]:
score = model.evaluate(x_test, y_test, verbose=0)  #evalute the model
print('Test loss:', score[0])  #gives the loss in testing
print('Test accuracy:', score[1])   #gives the accuracy of model

Test loss: 0.6980320811271667
Test accuracy: 0.8421000242233276


# General User Interface

Here we will create our interface where you can write any digit and get the predicted output

In [ ]:
from keras.models import load_model
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image
import numpy as np
model = load_model('mnist.h5')
def predict_digit(img):
    
    img = img.resize((28,28))  #resize image to 28x28 pixels
    img = img.convert('L')    #convert rgb to grayscale
    img = np.array(img)
    img = img.reshape(1,28,28,1)    #reshaping to support our model input and normalizing
    img = img/255.0 
    res = model.predict([img])[0]    #predicting the class
    
    return np.argmax(res), max(res)

class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        self.x = self.y = 0
        self.canvas = tk.Canvas(self, width=300, height=300, bg = "white", cursor="cross")   # Creating elements
        self.label = tk.Label(self, text="Thinking..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text = "Recognise", command =  self.classify_handwriting) 
        self.button_clear = tk.Button(self, text = "Clear", command = self.clear_all)
        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=W, )
        self.label.grid(row=0, column=1,pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)
        self.canvas.bind("<B1-Motion>", self.draw_lines)  #self.canvas.bind("<Motion>", self.start_pos)
        
    def clear_all(self):
        self.canvas.delete("all")
        
    def classify_handwriting(self):
        HWND = self.canvas.winfo_id() # get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND) # get the coordinate of the canvas
        im = ImageGrab.grab(rect)
        digit, acc = predict_digit(im)
        self.label.configure(text= str(digit)+', '+ str(int(acc*100))+'%')
        
    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r=8
        self.canvas.create_oval(self.x-r, self.y-r, self.x + r, self.y + r, fill='black')
app = App()
mainloop()